In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
import math

data = pd.read_csv("dataAfterPOS.csv")
data

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,word_count
0,Italy,aromas tropical fruit broom brimstone herb pal...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24
1,Portugal,ripe fruity wine firm tannins juicy red berry ...,Avidagos,87,15.000000,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39
2,US,tart snappy flavors lime flesh dominate green ...,unknown,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28
3,US,pineapple rind lemon pith orange blossom aroma...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33
4,US,much regular bottling rough tannic rustic eart...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41
5,Spain,blackberry typical navarran whiff green herbs ...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,unknown,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,42
6,Italy,bright informal aromas candied berry white pep...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,unknown,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,31
7,France,dry restrained wine offers profusion acidity f...,unknown,87,24.000000,Alsace,Alsace,unknown,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,21
8,Germany,savory thyme notes accent sunnier flavors pres...,Shine,87,12.000000,Rheinhessen,unknown,unknown,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,24
9,France,great depth flavor fresh apple pear fruits tou...,Les Natures,87,27.000000,Alsace,Alsace,unknown,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,30


# pre processing

In [2]:
#remove punctuation
data['description'] = data['description'].str.replace('[^\w\s]','')

In [3]:
#remove most frequent words, not influent bcs very frequent
freq = pd.Series(' '.join(data['description']).split()).value_counts()[:10]
freq

wine       72465
flavors    56228
fruit      40686
palate     33691
aromas     32164
acidity    31672
finish     29689
tannins    26477
cherry     23568
ripe       23310
dtype: int64

In [4]:
freq = list(freq.index)
data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [5]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

In [6]:
unfreq = pd.Series(' '.join(data["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
print(count)   #elementi poco frequenti sotto il 10 sono spesso typos/parole unite o robe brutte

30993


In [7]:
unfreqList = unfreq[-count:]
unfreqList

applecider           9
fields               9
innate               9
finetextured         9
whitetea             9
awaits               9
embers               9
mingling             9
sarsparilla          9
tapioca              9
backend              9
re                   9
welter               9
branded              9
providing            9
dor                  9
darkroast            9
southernmost         9
countywide           9
cherryblossom        9
volk                 9
predinner            9
danish               9
dna                  9
calaveras            9
elena                9
rosenblum            9
gonet                9
acidrich             9
cs                   9
                    ..
spicescurry          1
marchand             1
travelsize           1
worldlike            1
fruitmangomarries    1
floralrose           1
pomino               1
alberdi              1
mintandtomato        1
salineinflected      1
europeanstyled       1
citrusyclean         1
plumsauce  

In [8]:
unfreq = dict(unfreqList)  #con dictionary is veeeeeeeeeeeery fasteroni
data["description"] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

# TF IDF  fulltext

In [9]:
fullText = pd.Series(' '.join(data['description']).split()).value_counts()
fullText

black             22571
notes             16849
spice             16702
red               16189
rich              15706
fresh             15107
oak               15031
nose              14384
dry               13920
berry             13718
plum              12456
soft              12181
fruits            12165
apple             11360
texture           11358
blend             11323
blackberry        11094
sweet             11071
crisp             10990
white             10788
drink             10172
dark              10136
light             10076
citrus             9965
cabernet           9572
vanilla            9535
bright             9114
full               9092
good               8641
pepper             8640
                  ...  
outrageous           10
freshfruit           10
stargazer            10
thienpont            10
unexciting           10
planted              10
scotch               10
plane                10
wellhewn             10
music                10
pictureperfect  

In [10]:
tfData = {}
for idx, row in data.iterrows(): 
    tfData[idx] = pd.Series(row['description'].split()).value_counts();

In [11]:
tf1 = {}
for idx in tfData.keys():
    tf1[idx] = {}
    for word in tfData[idx].keys():
        tf1[idx][word] = tfData[idx][word] / fullText[word]


In [12]:
docCount = dict.fromkeys(fullText.keys(),0)
idfDict1 = {}
N = len(data)                                #numero di documenti #dizionario contenente tutte le parole presenti 
idfData = dict(tfData)
for el in idfData:     #per ogni descrizione
    for word,val in idfData[el].items():
        docCount[word] = docCount[word] + 1
        
for word in fullText.keys():
    idfDict1[word] = math.log10(N/docCount[word])  

In [13]:
#tf-idf
tfIdf1 = {}
for idx in tfData.keys():
    tfIdf1[idx] = {}
    for word in tfData[idx].keys():
        val = tf1[idx][word] * idfDict1[word]
        tfIdf1[idx][word] = val
tfIdf1[0]

{'citrus': 0.00010940615033478526,
 'herb': 0.0002350633883390736,
 'dried': 0.0005572701847069409,
 'broom': 0.016706767017015508,
 'apple': 9.272510825028737e-05,
 'tropical': 0.00048714719019093375,
 'expressive': 0.004275096725264226,
 'brisk': 0.000982947349387553,
 'brimstone': 0.0605497568141212,
 'sage': 0.0012048348388419927}